# Adicionando funções externas utilizando LangChain

LangChain é um framework para criação de aplicações de IA e naturalmente ele possui ferramentas para facilitar a criação de funções externas para serem passadas a api da OpenAI. Para a utilização do LangChain, será necessário entendermos brevemente uma outra biblioteca de Python chamada pydantic, uma bilioteca para validação de dados que facilita a construção de estruturas de dados mais robustas.

## Introduzindo pydantic

### Como criamos uma estrutura nova sem pydantic

Sem utilizar pydantic, podemos criar uma estrutura de dados utilizando funções da seguinte forma:

In [8]:
class Pessoa:
    def __init__(self, nome: str, idade: int, peso: float):
        self.nome = nome
        self.idade = idade
        self.peso = peso

Neste caso, criamos uma função que representa uma pessoa e tem os seguintes atributos: nome, idade e peso.

In [9]:
thomas = Pessoa('Thomas', 18, 52)

In [10]:
thomas.nome

'Thomas'

Podemos setar os atributos com qualquer tipo de dado, pois o Python por padrão não faz checagem de tipos.

In [12]:
thomas = Pessoa('Thomas', 18, '52')
thomas

In [13]:
thomas.peso

'52'

### Como criamos uma estrutura nova usando pydantic

A sintaxe de pydantic acaba sendo bem mais simples para a criação de classes de dados, ao compararmos com a criação de classes comuns de Python. Nela, temos que cuidar com a definição do tipo de cada atributo, pois eles serão utilizados para validar se os dados fornecidos estão corretos.

In [15]:
from pydantic import BaseModel

class pyPessoa(BaseModel):
    nome: str
    idade: int
    peso: float

In [16]:
adriano = pyPessoa(nome='Adriano', idade=32, peso=68)
adriano

pyPessoa(nome='Adriano', idade=32, peso=68.0)

In [17]:
adriano.nome

'Adriano'

O interessante é vermos que pydantic fornece uma validação automática de dados. Isso garante uma integridade muito maior em aplicações mais complexas.

In [18]:
adriano = pyPessoa(nome='Adriano', idade=32, peso='jhflj')

ValidationError: 1 validation error for pyPessoa
peso
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='jhflj', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [19]:
adriano = pyPessoa(nome='Adriano', idade=32, peso='68')
adriano.peso

68.0

Podemos fazer um nesting de classes de pydantic, onde uma classe de dados recebe como input outra classe de pydantic.

In [20]:
from typing import List

class pyAsimovTeam(BaseModel):
    funcionarios: List[pyPessoa]

pyAsimovTeam(funcionarios= [pyPessoa(nome='Adriano', idade=32, peso=68)])

pyAsimovTeam(funcionarios=[pyPessoa(nome='Adriano', idade=32, peso=68.0)])

E a validação continua fucnionando mesmo com esta estrutura de nesting.

In [21]:
pyAsimovTeam(funcionarios= [Pessoa(nome='Adriano', idade=32, peso=68)])

ValidationError: 1 validation error for pyAsimovTeam
funcionarios.0
  Input should be a valid dictionary or instance of pyPessoa [type=model_type, input_value=<__main__.Pessoa object at 0x000001F823C45E50>, input_type=Pessoa]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

## Utilizando pydantic para criação de tools da OpenAI

In [22]:
import json

def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )
    
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]


In [24]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
    celsius= 'celsius'
    fahrenheit= 'fahrenheit'

class ObterTemperaturaAtual(BaseModel):
    '''Obtem a temperatura atual de uma determinada localidade'''
    local: str= Field(description='O nome da cidade', examples=['São Paulo', 'Porto Alegre'])
    unidade: Optional[UnidadeEnum]

In [25]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

tool_temperatura = convert_pydantic_to_openai_function(ObterTemperaturaAtual)
tool_temperatura

C:\Users\maris\AppData\Local\Temp\ipykernel_11220\3747914274.py:3: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  tool_temperatura = convert_pydantic_to_openai_function(ObterTemperaturaAtual)


{'name': 'ObterTemperaturaAtual',
 'description': 'Obtem a temperatura atual de uma determinada localidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'description': 'An enumeration.',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['local']}}

## Adicionando função externa utilizando LangChain 

Agora que já sabemos criar funções que os modelos de llm entendam, podemos passar essas funções para os modelos de linguagem através da biblioteca langchain. Para isso temos duas formas, podemos utilizar o parâmetro functions ao chamar o método invoke dos chat_models:

In [30]:
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model='gpt-4o-mini')

resposta = chat.invoke('Qual é a temperatura de Porto Alegre', functions=[tool_temperatura])
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 93, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-bea9f439-138b-479c-9cde-a55c76d14264-0', usage_metadata={'input_tokens': 93, 'output_tokens': 25, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Ou podemos dar um bind e criar um novo componente de chat_model que terá acesso a função sempre que for chamado o invoke. Nestes dois casos, o modelo se comportará com o parâmetro "auto" de chamamento de função, ou seja, ele chamará a função quando necessitar, caso contrário se comportará como um modelo de linguagem normal.

In [31]:
chat = ChatOpenAI(model='gpt-4o-mini')
chat_com_func = chat.bind(functions=[tool_temperatura])
resposta = chat.invoke('Qual é a temperatura de Porto Alegre')
resposta

AIMessage(content='Desculpe, mas não consigo fornecer informações em tempo real, como a temperatura atual de Porto Alegre. Recomendo verificar um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 14, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-c0ff603a-f912-44d7-87c5-b45ceecb2847-0', usage_metadata={'input_tokens': 14, 'output_tokens': 42, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Podemos obrigar o modelo a sempre chamar uma função da seguinte forma:

In [33]:
resposta = chat.invoke(
    'Qual é a temperatura de Porto Alegre',
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
)
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 105, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1cc5ffa-2688-4c85-8699-e63d0be8dc40-0', usage_metadata={'input_tokens': 105, 'output_tokens': 7, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [34]:
resposta = chat.invoke(
    'Olá',
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
)
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 99, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a6ac6a7-df0b-4b34-9981-42a5753c7e92-0', usage_metadata={'input_tokens': 99, 'output_tokens': 6, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Adicionando a uma chain

Podemos adicionar agora este modelo com funções a um prompt e criar uma chain.

In [37]:
from langchain.prompts import ChatPromptTemplate as cpt

prompt = cpt.from_messages([
    ('system', 'Você é um assistente amigável chamado Isac'),
    ('user', '{input}')
])

chain = prompt | chat.bind(functions=[tool_temperatura])

In [38]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 98, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-4d394013-9f85-4419-bbb7-b295015bfe3b-0', usage_metadata={'input_tokens': 98, 'output_tokens': 9, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [39]:
chain.invoke({'input': 'Qual a temperatura em Floripa?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Florianópolis"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 104, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-296fe475-21c0-47e4-a3ff-05954165f2f3-0', usage_metadata={'input_tokens': 104, 'output_tokens': 19, 'total_tokens': 123, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})